In [2]:
import torch as th
from torch import nn
import numpy as np
import gym
import minerl

from src.minerl3161.hyperparameters import DQNHyperparameters
from src.minerl3161.models import DQNNet
from src.minerl3161.utils import np_dict_to_pt, sample_pt_state
from src.minerl3161.wrappers import mineRLObservationSpaceWrapper, MineRLDiscreteActionWrapper


/home/ayden/miniconda3/envs/minerl_env/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [1]:
import numpy as np
import pandas as pd
from kmodes.kprototypes import KPrototypes

In [3]:
env = gym.make("MineRLObtainDiamond-v0")
# hp = DQNHyperparameters()

In [4]:
env.action_space.sample()

OrderedDict([('attack', array(0)),
             ('back', array(0)),
             ('camera', array([ -82.908585, -146.40172 ], dtype=float32)),
             ('craft', 'torch'),
             ('equip', 'wooden_axe'),
             ('forward', array(0)),
             ('jump', array(1)),
             ('left', array(1)),
             ('nearbyCraft', 'wooden_axe'),
             ('nearbySmelt', 'none'),
             ('place', 'none'),
             ('right', array(1)),
             ('sneak', array(1)),
             ('sprint', array(1))])

In [3]:
action_agent = MineRLDiscreteActionWrapper(env)

In [5]:
done = False
obs = env.reset()

i =  0
CURR_ACTION = 1

while not done:
    if i % 10 == 0:
        CURR_ACTION = action_agent.action_space.sample()
    
    action = action_agent.action(CURR_ACTION)
    obs, reward, done, info = env.step(action)
    i += 1

Failed to take a step (error a bytes-like object is required, not 'NoneType'). Terminating episode and sending random observation, be aware. To account for this failure case in your code check to see if `'error' in info` where info is the info dictionary returned by the step function.


In [2]:
unwrapped_env = gym.make("MineRLObtainDiamondShovel-v0")


In [7]:
# real observation space
w = 16
h = 16
features = ['pov', 'stone_sword', 'stonecutter', 'stone_shovel']
stack = 4
n_actions = 8
device="cpu"
layer_size = 8

env = mineRLObservationSpaceWrapper(unwrapped_env, frame=stack, features=features, downsize_width=w, downsize_height=h)

obs_space = env.observation_space

In [8]:
net = DQNNet(obs_space, n_actions, layer_size=layer_size).to(device)

In [9]:


optim = th.optim.Adam(lr=0.01, params=net.parameters())

sample_input = sample_pt_state(obs_space, obs_space.spaces.keys(), batch=1)
sample_output = th.rand((1, n_actions))

nn_output = net(sample_input)

loss = nn.functional.mse_loss(nn_output, sample_output)

optim.zero_grad()
loss.backward()
optim.step()

In [16]:
net(sample_input)

tensor([[0.4081, 1.0331, 0.8548, 0.6891, 1.1998, 1.2111, 1.2101, 0.8098]],
       grad_fn=<AddBackward0>)